In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [2]:
def load_json(file_path):
    """
    Charge un fichier JSON et retourne son contenu.
    
    Args:
        file_path (str): Chemin du fichier JSON
    
    Returns:
        dict: Contenu du fichier JSON
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Chemins vers les fichiers JSON
files = {
    "fr": "../data/questions_fr.json",
    "en": "../data/questions_en.json",
    "ar": "../data/questions_ar.json"
}

In [3]:
data = {lang: load_json(path) for lang, path in files.items()}

In [4]:
def extract_questions_answers(data, lang="fr"):
    """
    Extrait les questions et réponses d'un fichier de données JSON.
    
    Args:
        data (dict): Données du chatbot
        lang (str): Langue (fr/en/ar)
        
    Returns:
        tuple: Liste des questions et des réponses
    """
    questions = []
    answers = []
    
    for item in data[lang].get('questions', []):
        questions.append(item['question'])
        answers.append(item['answer'])
    
    return questions, answers

In [5]:
questions_fr, answers_fr = extract_questions_answers(data, "fr")

In [8]:
def preprocess_data(questions, answers, max_len=20):
    """
    Prétraite les données pour le modèle en tokenisant et en préparant les séquences.
    
    Args:
        questions (list): Liste des questions
        answers (list): Liste des réponses
        max_len (int): Longueur maximale des séquences
        
    Returns:
        tuple: Séquences tokenisées et padées pour les questions et les réponses
    """
    tokenizer_q = Tokenizer()
    tokenizer_q.fit_on_texts(questions)
    
    # Initialisation du tokenizer pour les réponses
    tokenizer_a = Tokenizer()
    tokenizer_a.fit_on_texts(answers)
    
    # Ajout des tokens 'startseq' et 'endseq' dans le tokenizer des réponses
    tokenizer_a.word_index['startseq'] = len(tokenizer_a.word_index) + 1
    tokenizer_a.word_index['endseq'] = len(tokenizer_a.word_index) + 2
    tokenizer_a.index_word[tokenizer_a.word_index['startseq']] = 'startseq'
    tokenizer_a.index_word[tokenizer_a.word_index['endseq']] = 'endseq'

    # Convertir les questions et réponses en séquences
    seq_q = tokenizer_q.texts_to_sequences(questions)
    seq_a = tokenizer_a.texts_to_sequences(answers)
    
    # Pad les séquences
    seq_q_pad = pad_sequences(seq_q, maxlen=max_len, padding='post')
    seq_a_pad = pad_sequences(seq_a, maxlen=max_len, padding='post')
    
    # Ajouter un token de début et de fin pour les réponses
    seq_a_pad = np.array([np.append([tokenizer_a.word_index['startseq']], seq) for seq in seq_a_pad])
    seq_a_pad = np.array([np.append(seq, [tokenizer_a.word_index['endseq']]) for seq in seq_a_pad])
    
    return seq_q_pad, seq_a_pad, tokenizer_q, tokenizer_a


In [9]:
max_len = 20
seq_q_fr, seq_a_fr, tokenizer_q_fr, tokenizer_a_fr = preprocess_data(questions_fr, answers_fr, max_len)

In [10]:
def build_seq2seq_model(input_dim, output_dim, max_len):
    """
    Crée un modèle Seq2Seq simple avec LSTM.
    
    Args:
        input_dim (int): Taille du vocabulaire d'entrée
        output_dim (int): Taille du vocabulaire de sortie
        max_len (int): Longueur maximale des séquences
    
    Returns:
        Model: Modèle Keras
    """
    # Entrée pour les questions
    encoder_inputs = Input(shape=(max_len,))
    encoder = LSTM(256, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    
    # Sauvegarde des états pour la partie décodeur
    encoder_states = [state_h, state_c]
    
    # Entrée pour les réponses
    decoder_inputs = Input(shape=(max_len,))
    decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(output_dim, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Modèle final
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
input_dim = len(tokenizer_q_fr.word_index) + 1
output_dim = len(tokenizer_a_fr.word_index) + 1

In [12]:
model = build_seq2seq_model(input_dim, output_dim, max_len)

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 20)